In [2]:
#CARGAR LAS BASES DE DATOS
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
os.chdir('C:/Users/Nacha/OneDrive/Desktop/Anaconda/TP4')

In [3]:
# PARTE 1 
# EJERCICIO 2 
data_2024 = pd.read_excel('usu_hogar_T124.xlsx')
data_2004 = pd.read_stata('Hogar_t104.dta')